<a href="https://colab.research.google.com/github/AnshuKamath/DB-Analytics-Assignment/blob/main/Section1_R_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y r-base

In [ ]:
# Install required R packages
!R -e "install.packages(c('RSQLite', 'DBI', 'dplyr', 'ggplot2'), repos='https://cran.rstudio.com/')"

In [ ]:
# Load the R extension
%load_ext rpy2.ipython

In [ ]:
# Clone your GitHub repository to access the dataset
!git clone https://github.com/AnshuKamath/DB-Analytics-Assignment.git